In [40]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.dynamicframe import DynamicFrame

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
sys.argv += ["--JOB_NAME", "Demo"]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
args = getResolvedOptions(sys.argv, ['JOB_NAME'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
args

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'job_bookmark_option': 'job-bookmark-disable', 'job_bookmark_from': None, 'job_bookmark_to': None, 'JOB_ID': None, 'JOB_RUN_ID': None, 'SECURITY_CONFIGURATION': None, 'encryption_type': None, 'enable_data_lineage': None, 'RedshiftTempDir': None, 'TempDir': None, 'JOB_NAME': 'Demo'}

In [16]:
sc = spark.sparkContext

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
gc = GlueContext(sc)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/home/glue_user/spark/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(

In [20]:
job = Job(gc)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
job.init(args['JOB_NAME'], args)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Reading Data From AWS Glue Catalog (or S3)

In [24]:
# From Glue Catalog
df1 = gc.create_dynamic_frame.from_catalog(database="sales", table_name="items")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
df1.show(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{"item_id": "GWA1BUVW", "item_desc": "Item ASW8X", "start_date": "2008-03-19", "end_date": "2010-05-09", "price": 324.02, "item_class": "Beauty", "item_category": "Literature"}

In [28]:
# From S3
df2 = gc.create_dynamic_frame.from_options(
    connection_type = "s3",
    connection_options = {"paths": ["s3://shivchoudhury-datasets/customers/"]},
    format = "csv",
    format_options = {"withHeader": True}
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
df2.show(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{"CUSTOMER_ID": "WUWB8N9A", "SALUTATION": "Prof.", "FIRST_NAME": "Morgan", "LAST_NAME": "Garcia", "BIRTH_DAY": "04", "BIRTH_MONTH": "11", "BIRTH_YEAR": "1980", "BIRTH_COUNTRY": "Brazil", "EMAIL_ADDRESS": "morgan.garcia@mail.com"}

#### Transformation

In [32]:
df = df1.toDF()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/home/glue_user/spark/python/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")

In [34]:
df.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+----------+----------+----------+------+----------+-------------+
| item_id| item_desc|start_date|  end_date| price|item_class|item_category|
+--------+----------+----------+----------+------+----------+-------------+
|GWA1BUVW|Item ASW8X|2008-03-19|2010-05-09|324.02|    Beauty|   Literature|
|IXFQVLF1|Item SC253|2016-08-06|2018-12-28|281.07|  Clothing|    Furniture|
+--------+----------+----------+----------+------+----------+-------------+
only showing top 2 rows

In [35]:
filtered_df = df.filter(df["item_class"] == "Beauty")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
filtered_df.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+----------+----------+----------+------+----------+-------------+
| item_id| item_desc|start_date|  end_date| price|item_class|item_category|
+--------+----------+----------+----------+------+----------+-------------+
|GWA1BUVW|Item ASW8X|2008-03-19|2010-05-09|324.02|    Beauty|   Literature|
|P6UWDXZP|Item 20DLN|2019-04-05|2021-11-30|485.21|    Beauty|   Literature|
+--------+----------+----------+----------+------+----------+-------------+
only showing top 2 rows

In [41]:
filtered_dynamic_frames = DynamicFrame.fromDF(filtered_df, gc, "test")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
filtered_dynamic_frames.show(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{"item_id": "GWA1BUVW", "item_desc": "Item ASW8X", "start_date": "2008-03-19", "end_date": "2010-05-09", "price": 324.02, "item_class": "Beauty", "item_category": "Literature"}

#### Writing Data

In [46]:
gc.write_dynamic_frame.from_options(
    frame = filtered_dynamic_frames,
    connection_type = "s3",
    connection_options = {"path": "s3://shivchoudhury-datasets/customers-dynamic-frame/"},
    format = "parquet"
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
job.commit() # Mark job as completed

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…